In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('src/'))
if module_path not in sys.path:
    sys.path.append(module_path)
import my_app

my_app.my_app()


In [ ]:
from initial_request import get_initial_info
import os
from dotenv import load_dotenv
import helpers
from create_n_iterations import create_n_iterations
from assign_peer_review_by_group import assign_peer_review_by_group

load_dotenv() 

URL = os.getenv('API_INSTANCE')
KEY = os.getenv('API_TOKEN')
COURSE_ID = os.getenv('COURSE_ID')
GRAPH_URL = f"{URL}/api/graphql"

canvas = helpers.create_instance(URL, KEY)

course, assignments, group_sets, users = get_initial_info(GRAPH_URL, COURSE_ID, KEY)

In [ ]:
# for an assignment, determine if has peer reviews *uses returned val from dropdown (assignment_id)
# if there are peer reviews -> give option to delete (with warning - cannot undo)

ASSIGNMENT_ID = 1236302
# now assign peer reviews
_course = canvas.get_course(COURSE_ID)
_assignment = _course.get_assignment(ASSIGNMENT_ID)



In [ ]:

matched_group_category = helpers._return_single_dict_match(group_sets, "_id", '49736') 
#simple_groups_list = helpers._simplify_group_dicts(matched_group_category)
#submission_status = get_submission_status_per_user(_assignment)


In [ ]:
assign_peer_review_by_group(_assignment, simple_groups_list, 3)  

In [ ]:
simple_groups_list

def get_user_list_from_groups(simple_group_list):
    groups = []
    
    for i in simple_group_list:
        users = [j.get('canvas_id') for j in i.get('members')]
        groups.append(users)
        
    return(groups)

get_user_list_from_groups(simple_groups_list)

In [ ]:
from helpers import _return_single_dict_match, _simplify_group_dicts, _matches_dict_key_val_list, _matches_dict_key_val


def _delete_all_assignment_peer_reviews(assignment):
    submissions = assignment.get_submissions()
    
    for i in submissions:

        submission_pr = i.get_submission_peer_reviews()

        for j in submission_pr:

            delete_reviewer = j.__dict__.get("assessor_id")
            i.delete_submission_peer_review(delete_reviewer)
            
    print(f"Deleted all peer reviews for {assignment.name}")

def _assign_user_submission_reviews(submission, reviewers):
    # remove any current reviews for submission
    # assign new reviews
    #submission = assignment.get_submission(reviewee)
    
    submission = submission.get("submission")
    
    for i in reviewers:
        submission.create_submission_peer_review(i)
        
def _get_submission_status_per_user(assignment):
    submissions = assignment.get_submissions()
    
    submission_details = []
    
    for i in submissions:
        new_dict = {"user_id": i.user_id,
        "submission_id": i.id,
        "submission_missing": i.missing,
        "submission": i}
        
        submission_details.append(new_dict)
        
    return(submission_details)


    



In [ ]:
#assign_peer_review_by_group

def assign_peer_review_by_group(assignment, simple_groups_list, n_reviews):
    
    # get all submissions
    try:
        submission_details = _get_submission_status_per_user(assignment)
        print(f"Accessed submissions for {assignment.name}")
    except Exception as err:
        print(f"Error getting submission details: {err}")

    try:
        _delete_all_assignment_peer_reviews(assignment)
    except Exception as err:
        print(f"Error in deleting current reviews: {err}")

    
    for i in simple_groups_list: 
        # uses canvas submission so need user_id to be int
        
        try:
            print(f"Accessing submissions for group: {i.get('group_name')}")
            group_users = [int(j.get("canvas_id")) for j in i.get("members")]
            submission_by_group = [j for j in submission_details if _matches_dict_key_val_list(j, "user_id", group_users)]
        
        except Exception as err:
            print(f"Error in group {i.get('group_name')}: {err}")
            
        #complete_submissions = [k for k in submission_by_group if _matches_dict_key_val(k, "submission_missing": False)]
        # what if NO submission(?) I think this will assign regardless
        list_dict, user_review_dict = create_n_iterations(group_users, n_reviews)

        for k in user_review_dict:
            reviewee = list(k.keys())[0]
            reviewers = list(k.values())[0]

            print(f"{reviewee} will be reviewed by: {reviewers}")

            submission = _return_single_dict_match(submission_by_group, "user_id", reviewee)
            try:
                _assign_user_submission_reviews(submission, reviewers)
            except Exception as err:
                print(f"Error in assigning reviews: {err}")

  

In [ ]:
#print(list(zip(*[lista, listb, listc])))
for i in zip(*test.values()):
    j = list(i) 
    first = j[0]
    rest = j[1:]
    
    print(f'{first}: {rest}')

In [ ]:
def assign_peer_reviews(users, n_reviews):
    
    list_dict, user_review_dict = create_n_iterations(users, n_reviews)
    
    for i in user_review_dict:
        reviewee = list(i.keys())[0]
        reviewers = list(i.values())[0]
        
        print(f"{reviewee} is reviewed by: {reviewers}")
        
        _assign_user_submission_reviews(submission, reviewee, reviewers)
